In [0]:
%sql
-- Check the current schema
SELECT current_database();

  CREATE TABLE Employee (
    EmployeeID INT,
    Name VARCHAR(50),
    Department VARCHAR(50),
    Salary INT,
    JoiningDate DATE
);

INSERT INTO Employee (EmployeeID, Name, Department, Salary, JoiningDate) VALUES
(1, 'Alice', 'Sales', 70000, '2021-03-15'),
(2, 'Bob', 'Sales', 68000, '2022-04-20'),
(3, 'Charlie', 'Marketing', 72000, '2020-07-30'),
(4, 'David', 'Marketing', 75000, '2021-11-25'),
(5, 'Eve', 'Sales', 69000, '2020-02-10'),
(6, 'Frank', 'HR', 66000, '2019-05-15'),
(7, 'Grace', 'HR', 64000, '2021-06-10'),
(8, 'Hannah', 'Finance', 73000, '2022-08-19'),
(9, 'Ian', 'Finance', 71000, '2020-03-05'),
(10, 'Jack', 'Sales', 78000, '2023-01-10'),
(11, 'Kara', 'Marketing', 80000, '2022-05-05'),
(12, 'Liam', 'Finance', 72000, '2021-01-30');



---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4136628984228659>:9
      7     display(df)
      8     return df
----> 9   _sqldf = ____databricks_percent_sql()
     10 finally:
     11   del ____databricks_percent_sql

File <command-4136628984228659>:5, in ____databricks_percent_sql()
      3 import base64
      4 spark.sql(base64.standard_b64decode("LS0gQ2hlY2sgdGhlIGN1cnJlbnQgc2NoZW1hClNFTEVDVCBjdXJyZW50X2RhdGFiYXNlKCk=").decode())
----> 5 spark.sql(base64.standard_b64decode("YWNjb3JkaW5nIHRvIGRhdGEgYnJpY2tzIGNyZWF0ZSBhIGNvZGUgb24gQ1JFQVRFIFRBQkxFIEVtcGxveWVlICgKICAgIEVtcGxveWVlSUQgSU5ULAogICAgTmFtZSBWQVJDSEFSKDUwKSwKICAgIERlcGFydG1lbnQgVkFSQ0hBUig1MCksCiAgICBTYWxhcnkgSU5ULAogICAgSm9pbmluZ0RhdGUgREFURQop").decode())
      6 df = spark.sql(base64.standard_b64decode("SU5TRVJUIElOVE8gRW1wbG95ZWUgKEVtcGxveWVlSUQsIE5hbWUsIERlcGFydG1lbnQsIFNhbGFyeSwgSm9pbmluZ0RhdGUpIFZBTF

In [0]:
%sql
SELECT 
    Department,
    COUNT(EmployeeID) AS TotalEmployees,
    AVG(Salary) AS AverageSalary
FROM Employee
GROUP BY Department;



Department,TotalEmployees,AverageSalary
Sales,4,71250.0
HR,2,65000.0
Finance,3,72000.0
Marketing,3,75666.66666666667


In [0]:
%sql
SELECT 
    EmployeeID,
    Name,
    Department,
    Salary,
    RANK() OVER (PARTITION BY Department ORDER BY Salary DESC) AS SalaryRank
FROM Employee;


EmployeeID,Name,Department,Salary,SalaryRank
8,Hannah,Finance,73000,1
12,Liam,Finance,72000,2
9,Ian,Finance,71000,3
6,Frank,HR,66000,1
7,Grace,HR,64000,2
11,Kara,Marketing,80000,1
4,David,Marketing,75000,2
3,Charlie,Marketing,72000,3
10,Jack,Sales,78000,1
1,Alice,Sales,70000,2


In [0]:
%sql
WITH DeptSalaryRank AS (
    SELECT 
        EmployeeID,
        Name,
        Department,
        Salary,
        RANK() OVER (PARTITION BY Department ORDER BY Salary DESC) AS SalaryRank
    FROM Employee
)
SELECT 
    EmployeeID,
    Name,
    Department,
    Salary
FROM DeptSalaryRank
WHERE SalaryRank = 1;


EmployeeID,Name,Department,Salary
8,Hannah,Finance,73000
6,Frank,HR,66000
11,Kara,Marketing,80000
10,Jack,Sales,78000


In [0]:
%sql
SELECT 
    EmployeeID,
    Name,
    Department,
    Salary,
    SUM(Salary) OVER (PARTITION BY Department ORDER BY Salary DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS CumulativeSalary
FROM Employee;


EmployeeID,Name,Department,Salary,CumulativeSalary
8,Hannah,Finance,73000,73000
12,Liam,Finance,72000,145000
9,Ian,Finance,71000,216000
6,Frank,HR,66000,66000
7,Grace,HR,64000,130000
11,Kara,Marketing,80000,80000
4,David,Marketing,75000,155000
3,Charlie,Marketing,72000,227000
10,Jack,Sales,78000,78000
1,Alice,Sales,70000,148000


In [0]:
%sql
WITH DeptAvgSalary AS (
    SELECT 
        Department,
        AVG(Salary) AS AvgSalary
    FROM Employee
    GROUP BY Department
)
SELECT 
    e.EmployeeID,
    e.Name,
    e.Department,
    e.Salary
FROM Employee e
JOIN DeptAvgSalary d ON e.Department = d.Department
WHERE e.Salary > d.AvgSalary;


EmployeeID,Name,Department,Salary
6,Frank,HR,66000
8,Hannah,Finance,73000
10,Jack,Sales,78000
11,Kara,Marketing,80000


In [0]:
%sql
WITH DeptMaxSalary AS (
    SELECT 
        Department,
        MAX(Salary) AS MaxSalary
    FROM Employee
    GROUP BY Department
)
SELECT 
    e.EmployeeID,
    e.Name,
    e.Department,
    e.Salary,
    (dm.MaxSalary - e.Salary) AS SalaryDifference
FROM Employee e
JOIN DeptMaxSalary dm ON e.Department = dm.Department;


EmployeeID,Name,Department,Salary,SalaryDifference
1,Alice,Sales,70000,8000
2,Bob,Sales,68000,10000
3,Charlie,Marketing,72000,8000
4,David,Marketing,75000,5000
5,Eve,Sales,69000,9000
6,Frank,HR,66000,0
7,Grace,HR,64000,2000
8,Hannah,Finance,73000,0
9,Ian,Finance,71000,2000
10,Jack,Sales,78000,0


In [0]:
%sql
SELECT 
    YEAR(JoiningDate) AS HireYear,
    COUNT(EmployeeID) AS EmployeesHired
FROM Employee
GROUP BY YEAR(JoiningDate)
ORDER BY HireYear;


HireYear,EmployeesHired
2019,1
2020,3
2021,4
2022,3
2023,1


In [0]:
%sql
WITH DeptSalaryRank AS (
    SELECT 
        EmployeeID,
        Name,
        Department,
        Salary,
        ROW_NUMBER() OVER (PARTITION BY Department ORDER BY Salary DESC) AS SalaryRank
    FROM Employee
)
SELECT 
    EmployeeID,
    Name,
    Department,
    Salary
FROM DeptSalaryRank
WHERE SalaryRank <= 2;


EmployeeID,Name,Department,Salary
8,Hannah,Finance,73000
12,Liam,Finance,72000
6,Frank,HR,66000
7,Grace,HR,64000
11,Kara,Marketing,80000
4,David,Marketing,75000
10,Jack,Sales,78000
1,Alice,Sales,70000


In [0]:
%sql
SELECT 
    EmployeeID,
    Name,
    Department,
    Salary,
    AVG(Salary) OVER (PARTITION BY Department ORDER BY Salary DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS RunningAvgSalary
FROM Employee;


EmployeeID,Name,Department,Salary,RunningAvgSalary
8,Hannah,Finance,73000,73000.0
12,Liam,Finance,72000,72500.0
9,Ian,Finance,71000,72000.0
6,Frank,HR,66000,66000.0
7,Grace,HR,64000,65000.0
11,Kara,Marketing,80000,80000.0
4,David,Marketing,75000,77500.0
3,Charlie,Marketing,72000,75666.66666666667
10,Jack,Sales,78000,78000.0
1,Alice,Sales,70000,74000.0


In [0]:
%sql
SELECT 
    EmployeeID,
    Name,
    Department,
    DATEDIFF(CURRENT_DATE, JoiningDate) / 365.25 AS TenureYears,
    RANK() OVER (PARTITION BY Department ORDER BY DATEDIFF(CURRENT_DATE, JoiningDate) / 365.25 DESC) AS TenureRank
FROM Employee;


EmployeeID,Name,Department,TenureYears,TenureRank
9,Ian,Finance,4.706366,1
12,Liam,Finance,3.800137,2
8,Hannah,Finance,2.250513,3
6,Frank,HR,5.514031,1
7,Grace,HR,3.441478,2
3,Charlie,Marketing,4.303901,1
4,David,Marketing,2.981520,2
11,Kara,Marketing,2.540726,3
5,Eve,Sales,4.772074,1
1,Alice,Sales,3.679671,2
